In [ ]:
# basic cherry-pick

from hamilton_protocols.utils import alpha_to_index
from adaptyv_lab import Protocol
import pandas as pd

lay_file = "C:\\Users\\Adaptyvbio\\dev\\hamilton-protocols\\src\\hamilton_protocols\\layouts\\cherry-pick.lay"
cherry_pick_file = "C:\\Users\\Adaptyvbio\\Documents\\MAPS\\CHP-004.csv"

mapping = pd.read_csv(cherry_pick_file)
src_plate_names = mapping["Source Plate"].unique()
protocol = Protocol.from_layout("test", lay_file, simulator_mode=False)

if protocol.deck is None:
    raise ValueError("No deck found in the layout file.")
print(protocol.deck.grid)

tip_rack = protocol.deck.get_tip_rack("E1")
src_plates = protocol.deck.get_plate_stack("F4")[: len(src_plate_names)]
dst_plates = protocol.deck.get_plate_stack("F3")[: len(src_plate_names)][::-1]
src_plate = protocol.deck.get_plate("C3")
dst_plate = protocol.deck.get_plate("C2")

tips = tip_rack.to_list()

protocol.initialize()

for src_plate_name in src_plate_names:
    protocol.grip_get(src_plates.pop())
    protocol.grip_place(src_plate, eject_tool=1)

    print(f"Processing source plate: {src_plate_name}")

    plate_df = mapping[mapping["Source Plate"] == src_plate_name]
    for row in plate_df.iterrows():
        source_well = row[1]["Source Well"]
        dest_well = row[1]["Destination Well"]
        print(f"Source: {source_well} in {src_plate_name} -> Dest: {dest_well}")

        protocol.transfer(
            src_plate[*alpha_to_index(source_well)],
            dst_plate[*alpha_to_index(dest_well)],
            tips.pop(0),
            25,
        )

    protocol.grip_get(src_plate)
    protocol.grip_place(dst_plates.pop())

protocol.grip_eject()


async for response in protocol.run():
    print(response)

In [ ]:
# fab remapping
from hamilton_protocols.protocols.dna import (
    fab_mapping_protocol,
    FAbMappingParams,
    FAbMappingProtocolParams,
)
from hamilton_protocols.utils import dataframe_to_base64
import pandas as pd

df_b64 = dataframe_to_base64(
    pd.read_csv("C:\\Users\\Adaptyvbio\\Documents\\MAPS\\CRD-002-012_fabs.csv")
)

protocol = fab_mapping_protocol(
    FAbMappingProtocolParams(plates=[FAbMappingParams(plate_id="11", csv_data=df_b64)])
)

async for response in protocol.run():
    pass

In [ ]:
# dna reconstitution
from hamilton_protocols.protocols.dna import (
    DNAReconstitutionParams,
    DNAPlateParams,
    dna_reconstitution_protocol,
)
from src.hamilton_protocols.utils import dataframe_to_base64
import pandas as pd

plates = [191, 187, 190, 188]

plate_data = {}
for plate in plates:
    plate_data[plate] = dataframe_to_base64(
        pd.read_csv(
            f"C:\\Users\\Adaptyvbio\\Documents\\MAPS\\P-TWI-0{plate}_hamilton_reconstitution_mappings.csv"
        )
    )


protocol = dna_reconstitution_protocol(
    DNAReconstitutionParams(
        plates=[
            DNAPlateParams(plate_id=f"{plate}", csv_data=data)
            for plate, data in plate_data.items()
        ]
    ),
    simulate=False,
)

async for response in protocol.run():
    print(response)

In [ ]:
# SPR

from adaptyv_lab import Protocol

protocol = Protocol.from_layout(
    "test",
    "C:\\Users\\Adaptyvbio\\dev\\hamilton-protocols\\src\\hamilton_protocols\\layouts\\bli.lay",
    simulator_mode=False,
)
if protocol.deck is None:
    raise ValueError("No deck found in the layout file.")

src_plate = protocol.deck.get_plate("E5")
dst_plate = protocol.deck.get_plate("E4")
reservoir = protocol.deck.get_reservoir("B2")
tips = protocol.deck.get_tip_rack("D2")
tip_stack = protocol.deck.get_tip_rack_stack("E3")
sample_plate = protocol.deck.get_plate("C3")
sample_plate_stack_src = protocol.deck.get_plate_stack("F1")
sample_plate_stack_dst = protocol.deck.get_plate_stack("F2")[::-1]

protocol.initialize()

# add buffer
# protocol.pickup_tips(tips)
# for q in [(0, 0), (0, 1), (1, 0), (1, 0)]:
#     for _ in range(4):
#         protocol.aspirate(reservoir, 50)
#         protocol.dispense(plate[q], 50, liquid_height=15)
# protocol.eject_tips(mode=1)

# # add samples
# for q in [(0, 0), (1, 0), (0, 1), (1, 1)]:
#     protocol.grip_get(tip_stack.pop()).grip_place(tips)
#     protocol.grip_get(sample_plate_stack_src.pop()).grip_place(sample_plate)

#     protocol.pickup_tips(tips)
#     protocol.aspirate(sample_plate, 8)
#     protocol.dispense(plate[q], 8, liquid_height=15, mix_cycles=2, mix_volume=10)
#     protocol.eject_tips(mode=2)

#     protocol.grip_get(sample_plate).grip_place(sample_plate_stack_dst.pop())
#     protocol.grip_get(tips).grip_place(tips, waste=True)

protocol.grip_eject()

async for response in protocol.run():
    print(response)

In [1]:
# dna dilution

from hamilton_protocols.protocols.dna import (
    DNADilutionProtocolParams,
    dna_dilution_protocol,
    DNADilutionSourcePlateParams,
    DNADilutionDestinationPlateParams,
)

protocol = dna_dilution_protocol(
    DNADilutionProtocolParams(
        plates=[
            DNADilutionDestinationPlateParams(
                plate_id="P-DIL-0153",
                source_plates=[
                    DNADilutionSourcePlateParams(
                        plate_id="P-REC-0105",
                        rows=8,
                        cols=6,
                    ),
                ],
            ),
            DNADilutionDestinationPlateParams(
                plate_id="P-DIL-0154",
                source_plates=[
                    DNADilutionSourcePlateParams(
                        plate_id="P-REC-0105",
                        rows=8,
                        cols=3,
                        source_well="A7",
                    ),
                    DNADilutionSourcePlateParams(
                        plate_id="P-REC-0106",
                        rows=3,
                        cols=1,
                    ),
                ],
            ),
        ]
    ),
    simulate=True,
)

['P-REC-0105', 'P-REC-0106']
Creating P-DIL-0153
  Source Plate: P-REC-0105 at index 0
    Source Well: A1
    Destination Well: A1
    Rows: 8
    Columns: 6
0 0
   1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
A                                      ·     ·     ·     ·     ·     ·   
B                                                                        
C                                      ·     ·     ·     ·     ·     ·   
D                                                                        
E                                      ·     ·     ·     ·     ·     ·   
F                                                                        
G                                      ·     ·     ·     ·     ·     ·   
H                                                                        
I                                      ·     ·     ·     ·     ·     ·   
J                                                                        
K                      

IndexError: pop from empty list

In [ ]:
from src.hamilton_protocols.api.log_analyzer import (
    analyze_all_logs,
    get_combined_analysis,
)

log_dir = "C:\\Program Files (x86)\\HAMILTON\\LogFiles"

logs = analyze_all_logs(log_dir)
stats = get_combined_analysis(logs)
print(stats)